In [11]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from time import time
from random import randint
from IPython.display import clear_output
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [29]:
pages=[str(i) for i in range(1,5)]
years_url=[str(i) for i in range(1994,2006)]

In [80]:
requests = 0
start_time= time()
movies = []
years = []
metascores = []
votes = []
gross = []
imdb_ratings = []
directors = []
genre = []

for year_url in years_url:
    response= r.get('https://www.imdb.com/search/title/?year='+ year_url +'&sort=num_votes,desc&ref_=adv_prv')

#       https://www.imdb.com/search/title/?release_date='+ year_url +'&sort=num_votes,asc&page='+page
    sleep(randint(3,6))
    4#afficher les info sur les requests
    requests += 1
    elapsed_time=time()- start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait=True)
    4# waring if code_status is #200
    if response.status_code !=200:
        warn('Request:{}; status code:{}'.format(requests,response.status_code))
        5#stopper la boucle si le nombre de requests >72
    if requests>72:
        warn('many requests,slow down')
        break
    soup = BeautifulSoup(response.text, 'lxml')
    movies1 = [i.text for i in soup.select('h3.lister-item-header a')]
    movies.append(movies1)
    year1 = [i.text for i in soup.select('h3.lister-item-header span.lister-item-year')]
    years.append(year1)
    metascores1 = [i.text for i in soup.select('span.metascore')]
    if metascores1 is None:
        metascores.append(0)
    else:
        metascores.append(metascores1)
    votes1 = [i.text for i in soup.select('p.sort-num_votes-visible :nth-child(2)')]
    votes.append(votes1)
    gross1 = [i.text for i in soup.select('p.sort-num_votes-visible :nth-child(5)')]
    gross.append(gross1)
    imdb_ratings1= [i.text for i in soup.select('div.inline-block strong')]
    imdb_ratings.append(imdb_ratings1)
    director = [i.text for i in soup.select('div.lister-item p:nth-child(5)')]
    for i in director:
        directors.append(i.split('|')[0])
    genres1 = [i.text for i in soup.select('span.genre')]
    genre.append(genres1)
    

Request: 12; Frequency: 0.14115596805041383 requests/s


# Code to Fetch gross and metascores 

In [183]:
#redeclarer les listes
movies_content=soup.find_all("div",class_="lister-item mode-advanced")
metascores = []
gross = []
start_time= time()
requests= 0
3#for every year between 1994-2005 i make a loop for every page between 1 and 4 + request get
for year_url in years_url:
    response= r.get('https://www.imdb.com/search/title/?year='+ year_url +'&sort=num_votes,desc&ref_=adv_prv')
    4#pause sur la boucle de 8 a 15 sec
    sleep(randint(3,6))
    4#afficher les info sur les requests
    requests += 1
    elapsed_time=time()- start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait=True)
    4# waring if code_status is #200
    if response.status_code !=200:
        warn('Request:{}; status code:{}'.format(requests,response.status_code))
        5#stopper la boucle si le nombre de requests >72
    if requests>72:
        warn('many requests,slow down')
        break
    6# extract HTML with Bs4
    page_html= BeautifulSoup(response.text, 'html.parser')
    mv_containers=page_html.find_all("div",class_="lister-item mode-advanced")
    8#loop for every container
    for container in movies_content:
#             imbd=[float(i.text) for i in page_html.select('div.inline-block.ratings-imdb-rating')]
#             print(imbd)
#             imbd_ratings.append(imbd)
#             print(imbd_ratings)
        m_score=container.find('span', class_="metascore")
        if m_score is None:
            metascores.append(0)
        else:
            metascores.append(int(m_score.text))
        gross1 = [i.text for i in container.select('p.sort-num_votes-visible :nth-child(5)')]
        if m_score is None:
            gross.append(0)
        else:
            gross.append(gross1)



Request: 12; Frequency: 0.16568578658529268 requests/s


In [190]:
Gross = []
for i in gross:
    if i != 0:
        for j in i:
            Gross.append(j)
    else:
        Gross.append(i)

In [195]:
df = pd.DataFrame()
df['Movie'] = [j for i in movies for j in i]
df['Genre'] = [j for i in genre for j in i]
df['Director'] = [i for i in directors]
df['Year'] = [j for i in years for j in i]
df['Metascore'] = [i for i in metascores]
df['Vote'] = [j for i in votes for j in i]
df['Gross'] = [i for i in Gross]
df['Imdb_Rating'] = [j for i in imdb_ratings for j in i]


In [196]:
df

,Movie,Genre,Director,Year,Metascore,Vote,Gross,Imdb_Rating
0,Les évadés,\nDrama,\n Director:\nFrank Darabont\n,(1994),80,"2,160,007",$28.34M,9.3
1,Pulp Fiction,"\nCrime, Drama",\n Director:\nQuentin Tarantino\n,(1994),94,"1,695,374",$107.93M,8.9
2,Forrest Gump,"\nDrama, Romance",\n Director:\nRobert Zemeckis\n,(1994),82,"1,662,826",$330.25M,8.8
3,Léon,"\nAction, Crime, Drama",\n Director:\nLuc Besson\n,(1994),64,"955,442",$19.50M,8.5
4,The Lion King,"\nAnimation, Adventure, Drama","\n Directors:\nRoger Allers, \nRob Minkoff\n",(1994),88,"871,912",$422.78M,8.5
5,Friends,"\nComedy, Romance","\n \n Stars:\nJennifer Aniston, ...",(1994–2004),0,"715,645",0,8.9
6,Dumb & Dumber,\nComedy,"\n Directors:\nPeter Farrelly, \nBobby Farr...",(1994),41,"335,078",$127.18M,7.3
7,The Mask,"\nComedy, Crime, Fantasy",\n Director:\nChuck Russell\n,(1994),56,"321,562",$119.94M,6.9
8,Speed,"\nAction, Adventure, Thriller",\n Director:\nJan de Bont\n,(1994),78,"309,956",$121.25M,7.2
9,Entretien avec un vampire,"\nDrama, Horror",\n Director:\nNeil Jordan\n,(1994),59,"279,267",$105.26M,7.6
